In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse.linalg import expm_multiply, expm
from scipy.sparse import diags

from os.path import join, dirname
import sys
sys.path.append(join(".", "..", ".."))
from ionq_circuit_utils import *

import sys
sys.path.append(join(".", "..", "..", ".."))
from utils import *

import json
import hashlib

import networkx as nx
from random import shuffle, seed

from braket.devices import LocalSimulator

from qiskit import QuantumCircuit, transpile

Truncated Fock space method circuit

In [ ]:
def get_fock_space_method_circuit(N, a, b, t, r, encoding, use_second_order_pf=False):
    # Real-space Schrodinger equation with potential f(x) = 0.5 * ax^2 + bx

    n = num_qubits_per_dim(N, encoding=encoding)
    dt = t / r
    
    instructions = []

    np.random.seed(int(t * r))

    if encoding == "one-hot":
        if use_second_order_pf:
            for _ in range(r):
                # Diagonal part of 0.5 * \hat{p}^2 and 0.5 * a\hat{x}^2
                for i in range(n):
                    instructions.append(get_rz(-0.25 * (1 + a) * (i + 1/2) * dt, i))

                # Off-diagonal part of 0.5 * \hat{p}^2 and off-diagonal part of 0.5 * a\hat{x} ^ 2
                for i in range(n-2):
                    instructions.append(get_rxx(0.25 * (a * np.sqrt((i + 1) * (i + 2)) - np.sqrt((i + 1) * (i + 2))) * dt, [i, i + 2]))
                    instructions.append(get_ryy(0.25 * (a * np.sqrt((i + 1) * (i + 2)) - np.sqrt((i + 1) * (i + 2))) * dt, [i, i + 2]))
                # b \hat{x}
                for i in range(n-1):
                    instructions.append(get_rxx(b * np.sqrt((i + 1) / 2) * dt, [i, i + 1]))
                    instructions.append(get_ryy(b * np.sqrt((i + 1) / 2) * dt, [i, i + 1]))

                # Diagonal part of 0.5 * \hat{p}^2 and 0.5 * a\hat{x}^2
                for i in range(n):
                    instructions.append(get_rz(-0.25 * (1 + a) * (i + 1/2) * dt, i))
        else:
            for _ in range(r):
                if np.random.rand() < 0.5:
                    # Diagonal part of 0.5 * \hat{p}^2 and 0.5 * a\hat{x}^2
                    for i in range(n):
                        instructions.append(get_rz(-0.5 * (1 + a) * (i + 1/2) * dt, i))

                    # Off-diagonal part of 0.5 * \hat{p}^2 and off-diagonal part of 0.5 * a\hat{x} ^ 2
                    for i in range(n-2):
                        instructions.append(get_rxx(0.25 * (a * np.sqrt((i + 1) * (i + 2)) - np.sqrt((i + 1) * (i + 2))) * dt, [i, i + 2]))
                        instructions.append(get_ryy(0.25 * (a * np.sqrt((i + 1) * (i + 2)) - np.sqrt((i + 1) * (i + 2))) * dt, [i, i + 2]))
                    # b \hat{x}
                    for i in range(n-1):
                        instructions.append(get_rxx(b * np.sqrt((i + 1) / 2) * dt, [i, i + 1]))
                        instructions.append(get_ryy(b * np.sqrt((i + 1) / 2) * dt, [i, i + 1]))
                else:
                    # b \hat{x}
                    for i in range(n-1)[::-1]:
                        instructions.append(get_rxx(b * np.sqrt((i + 1) / 2) * dt, [i, i + 1]))
                        instructions.append(get_ryy(b * np.sqrt((i + 1) / 2) * dt, [i, i + 1]))

                    # Off-diagonal part of 0.5 * \hat{p}^2 and off-diagonal part of 0.5 * a\hat{x} ^ 2
                    for i in range(n-2)[::-1]:
                        instructions.append(get_rxx(0.25 * (a * np.sqrt((i + 1) * (i + 2)) - np.sqrt((i + 1) * (i + 2))) * dt, [i, i + 2]))
                        instructions.append(get_ryy(0.25 * (a * np.sqrt((i + 1) * (i + 2)) - np.sqrt((i + 1) * (i + 2))) * dt, [i, i + 2]))

                    # Diagonal part of 0.5 * \hat{p}^2 and 0.5 * a\hat{x}^2
                    for i in range(n)[::-1]:
                        instructions.append(get_rz(-0.5 * (1 + a) * (i + 1/2) * dt, i))

    else:
        raise ValueError("Encoding not supported")

    return instructions


In [ ]:
def run_fock_space_method(N, dimension, encoding, num_time_points, t_vals, task_name,
           r, a, b, num_shots_z, num_shots_x, device, save_dir, use_real_machine, use_second_order_pf,
           qpu_job_ids_filename_z, qpu_job_ids_filename_x):

    assert dimension == 1, "Expected dimension is 1"
    assert encoding == "one-hot"
    
    n = num_qubits_per_dim(N, encoding)

    if use_real_machine:
        # Job ids for circuits without Hadamard layer
        job_ids_z = []
        # Job ids for circuits with Hadamard layer
        job_ids_x = []
    else:
        distributions_z = []
        distributions_x = []

    for i, t in enumerate(t_vals):
        
        print(f"Unitless time: {t : 0.3f}")

        # Construct the circuit
        instructions = []

        # Initial state preparation: start from root node
        instructions.append(get_rx(np.pi, 0))
        if t > 0:
            instructions += get_fock_space_method_circuit(N, a, b, t, r, encoding, use_second_order_pf)
        
        # Send the circuits without Hadamard layer
        if use_real_machine:
            
            # Create the job json and save it
            job = get_ionq_job_json(task_name, N, dimension, num_shots_z, device, encoding, instructions, use_native_gates=True)
            
            with open(join(save_dir, f"t_{i}", f"job_z.json"), "w") as f:
                json.dump(job, f, default=int)

            # Send the job and get the job id
            job_id = send_job(job)
            print("Job id:", job_id)
            job_ids_z.append(job_id)
        else:

            native_instructions, qubit_phase = get_native_circuit(n, instructions)
            one_qubit_gate_count, two_qubit_gate_count = get_native_gate_counts(native_instructions)
            print(f"1q gates: {one_qubit_gate_count}, 2q gates: {two_qubit_gate_count}")
            circuit = get_braket_native_circuit(native_instructions)
            for j in range(n * dimension):
                circuit.rz(j, -qubit_phase[j] * (2 * np.pi))

            # Get amplitudes
            bitstrings = [np.binary_repr(j, n)[::-1] for j in range(2 ** n)]
            circuit.amplitude(state=bitstrings)
            
            # Run on simulator
            task = device.run(circuit)
            amplitudes_res = task.result().values[0]
            distributions_z.append(np.abs(list(amplitudes_res.values())) ** 2)

            # dist = np.zeros(2 ** N)
            # for measurement in task.result().measurements:
            #     dist[bitstring_to_int(measurement)] += 1
            # dist /= num_shots_z
            # distributions_z.append(dist)

        # Measure in X basis
        instructions += get_hadamard_layer(n, dimension)

        if use_real_machine:
            
            # Create the job json and save it
            job = get_ionq_job_json(task_name, N, dimension, num_shots_x, device, encoding, instructions)
            
            with open(join(save_dir, f"t_{i}", f"job_x.json"), "w") as f:
                json.dump(job, f, default=int)
            # Send the job and get the job id
            job_id = send_job(job)
            print("Job id:", job_id)
            job_ids_x.append(job_id)

        else:
            native_instructions, qubit_phase = get_native_circuit(n, instructions)
            circuit = get_braket_native_circuit(native_instructions)
            for j in range(n * dimension):
                circuit.rz(j, -qubit_phase[j] * (2 * np.pi))

            # Get amplitudes
            bitstrings = [np.binary_repr(j, n)[::-1] for j in range(2 ** n)]
            circuit.amplitude(state=bitstrings)

            # Run on simulator
            task = device.run(circuit)
            amplitudes_res = task.result().values[0]
            distributions_x.append(np.abs(list(amplitudes_res.values())) ** 2)

            # dist = np.zeros(2 ** N)
            # for measurement in task.result().measurements:
            #     dist[bitstring_to_int(measurement)] += 1
            # dist /= num_shots_x
            # distributions_x.append(dist)

    if use_real_machine:
        print("Saving IonQ job ids")
        with open(join(save_dir, qpu_job_ids_filename_z), "w") as f:
            json.dump(job_ids_z, f)
            f.close()

        with open(join(save_dir, qpu_job_ids_filename_x), "w") as f:
            json.dump(job_ids_x, f)
            f.close()

    if not use_real_machine:
        return distributions_z, distributions_x

# Fock space method for real space dynamics

In [ ]:
DATA_DIR = "experiment_data"
TASK_DIR = "real_space_sim"

CURR_DIR = join("..", "..", "..", "..", DATA_DIR)
check_and_make_dir(CURR_DIR)
CURR_DIR = join(CURR_DIR, TASK_DIR)
check_and_make_dir(CURR_DIR)

print(CURR_DIR)

use_real_machine = False
if use_real_machine:
    device = "qpu.aria-1"
    print("Device:", device)
else:
    device = LocalSimulator()

    print(f"Using {device.name}")

In [ ]:
N = 5
dimension = 1
encoding = "one-hot"
n = num_qubits_per_dim(N, encoding)
codewords = get_codewords_1d(n, encoding, periodic=False)
bitstrings = get_bitstrings(N, dimension, encoding)

T = 5
r = 11
a = 2
b = -1/2

num_time_points = 21
t_vals = np.linspace(0, T, num_time_points)
num_shots_z = 500
num_shots_x = 1500

use_error_mitigation = True
# Use second-order or use randomized first-order
use_second_order_pf = False

experiment_info = {
    "N": N,
    "dimension": dimension,
    "encoding": encoding,
    "T": T,
    "num_time_points": num_time_points,
    "r": r,
    "a": a,
    "b": b,
    "num_shots_z": num_shots_z,
    "num_shots_x": num_shots_x,
    "use_error_mitigation": use_error_mitigation,
    "use_second_order_pf": use_second_order_pf
}

hash_str = hashlib.md5(json.dumps(experiment_info).encode("utf-8")).hexdigest()
SAVE_DIR = join(CURR_DIR, hash_str)
check_and_make_dir(SAVE_DIR)

print("Save dir:", SAVE_DIR)

with open(join(SAVE_DIR, "experiment_info.json"), "w") as f:
    json.dump(experiment_info, f)
    f.close()

for i in range(num_time_points):
    if not exists(join(SAVE_DIR, f"t_{i}")):
        mkdir(join(SAVE_DIR, f"t_{i}"))

qpu_job_ids_filename_z = 'job_ids_qpu_z.json'
qpu_job_ids_filename_x = 'job_ids_qpu_x.json'

Submit tasks

In [ ]:
if use_real_machine:
    run_fock_space_method(N, dimension, encoding, num_time_points, t_vals, 
                          TASK_DIR, r, a, b, num_shots_z, num_shots_x, device, SAVE_DIR, 
                          use_real_machine, use_second_order_pf, 
                          qpu_job_ids_filename_z, qpu_job_ids_filename_x)
else:
    sim_prob_z, sim_prob_x = run_fock_space_method(N, dimension, encoding, num_time_points, t_vals, 
                                                   TASK_DIR, r, a, b, num_shots_z, num_shots_x, device, SAVE_DIR, 
                                                   use_real_machine, use_second_order_pf, 
                                                   qpu_job_ids_filename_z, qpu_job_ids_filename_x)

Get data from completed tasks

In [ ]:
def get_results_fock_space_method(N, job_ids_path, use_error_mitigation=False):

    with open(job_ids_path, "r") as f:
        job_ids = json.load(f)
        f.close()

    freq = np.zeros((len(job_ids), 2 ** N))
    for i, job_id in enumerate(job_ids):
        freq[i] = get_ionq_single_job_result(job_id, range(2 ** N), use_error_mitigation)

    return freq

In [ ]:
ionq_freq_z = get_results_fock_space_method(N, join(SAVE_DIR, qpu_job_ids_filename_z))
ionq_freq_x = get_results_fock_space_method(N, join(SAVE_DIR, qpu_job_ids_filename_x))

Post processing and figures

In [ ]:
# Observable for each computational basis state
x_obs_vec = np.zeros(2 ** n)
p2_obs_vec_x = np.zeros(2 ** n)
p2_obs_vec_z = np.zeros(2 ** n)

for i in range(2 ** n):
    bitstring = np.binary_repr(i, n)[::-1]

    for j in range(N-1):
        if bitstring[j] == bitstring[j+1]:
            x_obs_vec[i] += np.sqrt((j+1))
        else:
            x_obs_vec[i] -= np.sqrt((j+1))
        
    for j in range(N-2):
        if bitstring[j] == bitstring[j+2]:
            p2_obs_vec_x[i] += np.sqrt((j+1) * (j+2))
        else:
            p2_obs_vec_x[i] -= np.sqrt((j+1) * (j+2))
    
    for j in range(N):
        if bitstring[j] == '1':
            p2_obs_vec_z[i] += (j + 1/2)


x_obs_vec /= np.sqrt(2)
p2_obs_vec_x *= (-0.5)



In [ ]:
x_obs_sim = np.zeros(num_time_points)
p2_obs_sim = np.zeros(num_time_points)

x_obs_ionq = np.zeros(num_time_points)
p2_obs_x_ionq = np.zeros(num_time_points)
p2_obs_z_ionq = np.zeros(num_time_points)
pos_observable_ionq = []
for i in range(num_time_points):
    x_obs_sim[i] = x_obs_vec @ sim_prob_x[i]
    x_obs_ionq[i] = x_obs_vec @ ionq_freq_x[i]

    p2_obs_sim[i] = p2_obs_vec_x @ sim_prob_x[i] + (p2_obs_vec_z[codewords] @ sim_prob_z[i][codewords]) / np.sum(sim_prob_z[i][codewords])
    p2_obs_x_ionq[i] = p2_obs_vec_x @ ionq_freq_x[i]
    p2_obs_z_ionq[i] = (p2_obs_vec_z[codewords] @ ionq_freq_z[i][codewords]) / np.sum(ionq_freq_z[i][codewords])

p2_obs_ionq = p2_obs_x_ionq + p2_obs_z_ionq

In [ ]:
x_obs_ionq_err = np.zeros(num_time_points)
kinetic_energy_err = np.zeros(num_time_points)

for i in range(num_time_points):
    x_obs_ionq_err[i] = np.sqrt(ionq_freq_x[i] @ (x_obs_vec - x_obs_ionq[i]) ** 2 / (num_shots_x - 1))

    # Sqrt of variance of mean for XX observables plus Z observables
    kinetic_energy_err[i] = np.sqrt(0.5 * ionq_freq_x[i] @ (p2_obs_vec_x - p2_obs_x_ionq[i]) ** 2 / (num_shots_x - 1) + \
                                    0.5 * ionq_freq_z[i][codewords] @ (p2_obs_vec_z[codewords] - p2_obs_z_ionq[i]) ** 2 / (num_shots_z - 1))


In [ ]:
def get_real_space_hamiltonian(N, a, b):
    p_sq = sum_h_z(N, [-0.5 * (j+1/2) for j in range(N)])
    J = np.zeros((N,N))
    for i in range(N-2):
        J[i, i+2] = -0.5 * np.sqrt((i+1)*(i+2))
    p_sq += (sum_J_xx(N, J) + sum_J_yy(N, J)) / 2

    J = np.zeros((N,N))
    for i in range(N-1):
        J[i, i+1] = np.sqrt((i+1))
    f_linear = b * (1/np.sqrt(2)) * (sum_J_xx(N, J) + sum_J_yy(N, J)) / 2

    J = np.zeros((N,N))
    for i in range(N-2):
        J[i, i+2] = 0.5 * np.sqrt((i+1)*(i+2))

    f_quadratic = 0.5 * a * (sum_h_z(N, [-0.5 * (j+1/2) for j in range(N)]) + (sum_J_xx(N, J) + sum_J_yy(N, J)) / 2)

    H = 0.5 * p_sq + f_linear + f_quadratic
    return H

In [ ]:
H_simulator = get_real_space_hamiltonian(N, a, b)
codewords = get_codewords_1d(n, encoding=encoding, periodic=False)

In [ ]:
psi_0 = np.zeros(2 ** N)
psi_0[codewords[0]] = 1

# Position observable
J = np.zeros((N,N))
for i in range(N-1):
    J[i, i+1] = np.sqrt((i+1))
x_hat = (1/np.sqrt(2)) * sum_J_xx(N, J)

x_sq = sum_delta_n(N, [(j+1/2) for j in range(N)])
J = np.zeros((N,N))
for i in range(N-2):
    J[i, i+2] = 0.5 * np.sqrt((i+1)*(i+2))
x_sq += sum_J_xx(N, J)

p_sq = sum_delta_n(N, [(j+1/2) for j in range(N)])
J = np.zeros((N,N))
for i in range(N-2):
    J[i, i+2] = -0.5 * np.sqrt((i+1)*(i+2))
p_sq += sum_J_xx(N, J)

num_time_points_ideal = num_time_points
psi = expm_multiply(-1j * H_simulator, psi_0, start=0, stop=T, num=num_time_points_ideal)

In [ ]:
x_obs_ham_ebd = np.zeros(num_time_points_ideal)
x2_obs_ham_ebd = np.zeros(num_time_points_ideal)
p2_obs_ham_ebd = np.zeros(num_time_points_ideal)

for i in range(num_time_points_ideal):
    x_obs_ham_ebd[i] = (np.conj(psi[i]) @ x_hat @ psi[i]).real
    x2_obs_ham_ebd[i] = (np.conj(psi[i]) @ x_sq @ psi[i]).real
    p2_obs_ham_ebd[i] = (np.conj(psi[i]) @ p_sq @ psi[i]).real

kinetic_energy_ham_ebd = 0.5 * p2_obs_ham_ebd
potential_energy_ham_ebd = 0.5 * a * x2_obs_ham_ebd + b * x_obs_ham_ebd

In [ ]:
# Analytical solution
expected_position_analytical = (b / a) * (np.cos(np.sqrt(a) * t_vals) - 1)

In [ ]:
with open("experiment_info.json", "w") as f:
    json.dump(experiment_info, f)
    f.close()

np.savez("data.npz", 
         expected_position_analytical=expected_position_analytical,
         ionq_freq_z=ionq_freq_z,               # Relative frequency in z basis (all 2^5 states; 3000 shots total for each time point)
         ionq_freq_x=ionq_freq_x,               # Relative frequency in x basis (all 2^5 states; 3000 shots total for each time point)
         x_obs_ham_ebd=x_obs_ham_ebd,           # <\hat{x}> for Hamiltonian embedding
         x_obs_sim=x_obs_sim,                   # <\hat{x}> for circuit simulator
         x_obs_ionq=x_obs_ionq,                 # <\hat{x}> for IonQ
         x_obs_ionq_err=x_obs_ionq_err,         # Standard error for <\hat{x}>; refer to notebook for calculation
         p2_obs_ham_ebd=p2_obs_ham_ebd,         # <\hat{p}^2> for Hamiltonian embedding
         p2_obs_sim=p2_obs_sim,                 # <\hat{p}^2> for circuit simulator
         p2_obs_ionq=p2_obs_ionq,               # <\hat{p}^2> for IonQ
         kinetic_energy_err=kinetic_energy_err) # std error for kinetic energy